In [1]:

from sandbox.specs import (
    MAP_REPO_TO_REQS_PATHS,
    MAP_REPO_VERSION_TO_SPECS_PY,
)
from sandbox import bash_session, docker_build

In [2]:
from datasets import load_dataset

dataset = load_dataset("lahirum/SWE_Experimental", split="train")

/home/lahiru-menik/miniconda3/envs/agentless/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAP_REPO_VERSION_TO_SPECS_PY[dataset[0]['repo']][dataset[0]['version']]

{'python': '3.9',
 'packages': 'requirements.txt',
 'install': 'python -m pip install -e .',
 'test_cmd': './tests/runtests.py --verbosity 2'}

In [5]:
import docker

def image_exists(name):
    client = docker.from_env()
    try:
        client.images.get(name)
        return True
    except docker.errors.ImageNotFound:
        return False
    except docker.errors.APIError as e:
        print(f"Docker API error: {e}")
        return False

In [ ]:
from commit import update
import os
import tempfile
from pathlib import Path
import pexpect

for i in range(1):
    commit_id = dataset[i]['base_commit']
    fail_to_passs = dataset[i]['FAIL_TO_PASS']
    version = dataset[i]['version']
    pass_to_pass = dataset[i]['PASS_TO_PASS']
    name = dataset[i]['instance_id'].split("__")[0]
    # print(name)
    update(name, commit_id)
    bash = bash_session.BashSession()
    bash.run_command(f"cp -r {name} testbed")
    host_dir = os.path.abspath(".") + "/testbed/" + name
    container_dir = "/app" 
    exist_check = image_exists(name + "_" + version)
    # exist_check = bash.run_command(f"docker images | grep -q '^{name+'_'+version}' && echo yes || echo no")
    if not exist_check:
        print(f"Building image {name}_{version}...")
        specs = MAP_REPO_VERSION_TO_SPECS_PY[dataset[i]['repo']][dataset[i]['version']]
        if "pip_packages" in specs:
            pip_packages = " ".join(specs["pip_packages"])
        else:
            pip_packages = ""
        if "pre_install" in specs:
            pre_install_commands = "\n".join(specs["pre_install"])
        else:
            pre_install_commands = ""
        dockerfile_str = docker_build.generate_dockerfile(
            python_version= specs["python"],
            pre_install= pre_install_commands,
            pip_packages=pip_packages,
            
        )
        
        tmpdir = tempfile.mkdtemp()
        Path(f"{tmpdir}/Dockerfile").write_text(dockerfile_str)
        print(f"Generated Dockerfile for {name}_{version}")
        bash.run_command(f"docker build -t {name}_{version} {tmpdir}")
        bash.logfile = open(f"{name}_{version}_dockerbuild.log", "w")
        bash.expect(pexpect.EOF, timeout=600)
    
    
    

django
DiGraph with 27538 nodes and 40138 edges
Checked out to 4fd3044ca0135da903a70dfb66992293f529ecf1
DiGraph with 27538 nodes and 40138 edges
Building image django_4.1...
Generated Dockerfile for django_4.1
